# Считаю фиксированные эффекты в международной торговле, чтобы потом строить на них регрессии

Несколько спецификаций модели:

* hs0 или hs2 (hs4 и hs6 слишком тяжёлые) (указываю имя файла)
* log(v) или log(v/v19) (plain_or_change)
* v или q (value_or_quantity)

P.S. hs2 пока тоже не получаются из-за большого числа фиксированных эффектов

In [1]:
source('fixed_effects_computation.R')

Warning message:
"пакет 'tidyverse' был собран под R версии 4.4.2"
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Присоединяю пакет: 'arrow'


Следующий объект скрыт от 'package:lubridate':

    duration


Следующий объект скрыт от 'package:utils':

    timestamp




## HS0

### Делаю вручную, потому что fixest неправильно считает фиксированные эффекты

In [2]:
df = read_parquet("../preprocessed_data/trade_hs0.parquet.gzip")

In [4]:
fe_regr = lm('log(v) ~ as.factor(t) + as.factor(paste(t, i)) + as.factor(paste(t, j))', df)
fe_regr


Call:
lm(formula = "log(v) ~ as.factor(t) + as.factor(paste(t, i)) + as.factor(paste(t, j))", 
    data = df)

Coefficients:
                   (Intercept)                as.factor(t)2021  
                     -2.626888                        6.794607  
              as.factor(t)2022  as.factor(paste(t, i))2020 AFG  
                      6.503311                        2.299408  
as.factor(paste(t, i))2020 AGO  as.factor(paste(t, i))2020 AIA  
                      4.723406                       -0.500371  
as.factor(paste(t, i))2020 ALB  as.factor(paste(t, i))2020 AND  
                      4.738112                        1.492163  
as.factor(paste(t, i))2020 ARE  as.factor(paste(t, i))2020 ARG  
                     10.756787                        9.325794  
as.factor(paste(t, i))2020 ARM  as.factor(paste(t, i))2020 ASM  
                      4.393368                       -0.111814  
as.factor(paste(t, i))2020 ATF  as.factor(paste(t, i))2020 ATG  
                     -0.77943

In [12]:
all_effects = data.frame(name=fe_regr$coefficients %>% names(), effect=fe_regr$coefficients)
all_effects %>% head()

,name,effect
,<chr>,<dbl>
(Intercept),(Intercept),-2.6268879
as.factor(t)2021,as.factor(t)2021,6.7946065
as.factor(t)2022,as.factor(t)2022,6.5033105
"as.factor(paste(t, i))2020 AFG","as.factor(paste(t, i))2020 AFG",2.2994076
"as.factor(paste(t, i))2020 AGO","as.factor(paste(t, i))2020 AGO",4.7234056
"as.factor(paste(t, i))2020 AIA","as.factor(paste(t, i))2020 AIA",-0.5003708


In [40]:
HandmadeFE = function(all_effects, filter){
    filtered_effects = all_effects %>% filter(grepl(filter, name))
    year_country_matrix = filtered_effects$name %>% strsplit("))") %>%
    do.call(rbind, .) %>% .[, 2] %>% 
    strsplit(" ")  %>% do.call(rbind, .)
    filtered_effects$year = year_country_matrix[, 1]
    filtered_effects$country = year_country_matrix[, 2]
    filtered_effects$value = filtered_effects$effect
    rownames(filtered_effects) = NULL
    filtered_effects %>% select(year, country, value)
    
}
a = HandmadeFE(all_effects, "t, i") 
a %>% write_parquet(., "../preprocessed_data/trade_hs0_export_fe_handmade.parquet.gzip")
b = HandmadeFE(all_effects, "t, j") 
b %>% write_parquet(., "../preprocessed_data/trade_hs0_import_fe_handmade.parquet.gzip")

year,country,value
<chr>,<chr>,<dbl>
2020,AFG,1.7377734
2020,AGO,2.9645562
2020,AIA,-2.0771210
2020,ALB,2.3899286
2020,AND,-0.5420130
2020,ARE,7.5141743
2020,ARG,4.0374437
2020,ARM,2.2008491
2020,ASM,-1.1817834


### Через fixest

In [2]:
ComputeFE("../preprocessed_data/trade_hs0.parquet.gzip", 
                 is_category_aggregated = F, 
                 value_or_quantity = "value",
                 plain_or_change = 'plain',
                 save_path = "../preprocessed_data/trade_hs0")

           Demeaning: 0.33s (iter: 4)
          Estimation: 0.09s
OLS estimation, Dep. Var.: log(v)
Observations: 71,789
Fixed-effects: t: 3,  export_fe: 678,  import_fe: 678,  bilateral_fe: 25,779
RMSE: 0.718774   Adj. R2: 0.954755


NOTE: The fixed-effects are not regular, they cannot be straightforwardly interpreted.



Fixed_effects coefficients
                            t export_fe import_fe bilateral_fe
Number of fixed-effects     3       678       678        25779
Number of references        0         3         3          451
Mean                    0.647      3.26     0.748      -0.0448
Standard-deviation       11.2      16.2      9.35          9.8

COEFFICIENTS:
  t:   2020  2021  2022
     -12.26 6.867 7.335
-----
  export_fe: 2020 _ ABW 2020 _ AFG 2020 _ AGO 2020 _ AIA 2020 _ ALB
                  30.57      19.46      17.29      21.04      26.42
                   
  ... 673 remaining
-----
  import_fe: 2020 _ ABW 2020 _ AFG 2020 _ AGO 2020 _ AIA 2020 _ ALB
                    -10      3.424      3.264    -0.9014     -1.147
                   
  ... 673 remaining
-----
  bilateral_fe: ABW _ ARE ABW _ AUT ABW _ BEL ABW _ BRB ABW _ CAN
                   -7.992    -12.49    -22.97         0    -12.02
                      
  ... 25,774 remaining
NULL


In [13]:
ComputeFE("../preprocessed_data/trade_hs0.parquet.gzip", 
                 is_category_aggregated = F, 
                 value_or_quantity = "quantity",
                 plain_or_change = 'plain',
                 save_path = "../preprocessed_data/trade_hs0")

NOTE: 423 observations removed because of infinite values (LHS: 423).



           Demeaning: 0.23s (iter: 5)
          Estimation: 0.09s
OLS estimation, Dep. Var.: log(q)
Observations: 71,366
Fixed-effects: export_fe: 678,  import_fe: 678,  bilateral_fe: 25,640
RMSE: 0.903216   Adj. R2: 0.949483


NOTE: The fixed-effects are not regular, they cannot be straightforwardly interpreted.



Fixed_effects coefficients
                        export_fe import_fe bilateral_fe
Number of fixed-effects       678       678        25640
Number of references            0         3          451
Mean                        0.665      2.13       -0.569
Standard-deviation           10.4      10.6         12.9

COEFFICIENTS:
  export_fe: 2020 _ ABW 2020 _ AFG 2020 _ AGO 2020 _ AIA 2020 _ ALB
                  9.721      3.983     -2.184     -27.71       3.52
                   
  ... 673 remaining
-----
  import_fe: 2020 _ ABW 2020 _ AFG 2020 _ AGO 2020 _ AIA 2020 _ ALB
                  2.307      6.749     0.4296     -5.227      24.86
                   
  ... 673 remaining
-----
  bilateral_fe: ABW _ ARE ABW _ AUT ABW _ BEL ABW _ BRB ABW _ CAN
                   -10.48    -29.39    -26.18    -8.407    -38.14
                      
  ... 25,635 remaining
NULL


In [14]:
ComputeFE("../preprocessed_data/trade_hs0.parquet.gzip", 
                 is_category_aggregated = F, 
                 value_or_quantity = "value",
                 plain_or_change = 'change',
                 save_path = "../preprocessed_data/trade_hs0")

           Demeaning: 0.23s (iter: 5)
          Estimation: 0.11s
OLS estimation, Dep. Var.: log_change
Observations: 71,789
Fixed-effects: export_fe: 678,  import_fe: 678,  bilateral_fe: 25,779
RMSE: 0.592473   Adj. R2: 0.47783


NOTE: The fixed-effects are not regular, they cannot be straightforwardly interpreted.



Fixed_effects coefficients
                        export_fe import_fe bilateral_fe
Number of fixed-effects       678       678        25779
Number of references            0         3          451
Mean                       0.0991    -0.186       0.0342
Standard-deviation           2.08       1.6         2.31

COEFFICIENTS:
  export_fe: 2020 _ ABW 2020 _ AFG 2020 _ AGO 2020 _ AIA 2020 _ ALB
                0.09362     0.2921      1.769      3.877     -1.546
                   
  ... 673 remaining
-----
  import_fe: 2020 _ ABW 2020 _ AFG 2020 _ AGO 2020 _ AIA 2020 _ ALB
                  2.347     0.9312    -0.7634    -0.4088      -1.47
                   
  ... 673 remaining
-----
  bilateral_fe: ABW _ ARE ABW _ AUT ABW _ BEL ABW _ BRB ABW _ CAN
                    4.471     1.789    0.7316     4.512   -0.8552
                      
  ... 25,774 remaining
NULL


In [15]:
ComputeFE("../preprocessed_data/trade_hs0.parquet.gzip", 
                 is_category_aggregated = F, 
                 value_or_quantity = "quantity",
                 plain_or_change = 'change',
                 save_path = "../preprocessed_data/trade_hs0")

NOTE: 647 observations removed because of NA and infinite values (LHS: 647).



           Demeaning: 0.25s (iter: 4)
          Estimation: 0.11s
OLS estimation, Dep. Var.: log_change
Observations: 71,142
Fixed-effects: export_fe: 678,  import_fe: 678,  bilateral_fe: 25,541
RMSE: 0.689779   Adj. R2: 0.462826


NOTE: The fixed-effects are not regular, they cannot be straightforwardly interpreted.



Fixed_effects coefficients
                        export_fe import_fe bilateral_fe
Number of fixed-effects       678       678        25541
Number of references            0         3          451
Mean                       0.0253   -0.0544       -0.176
Standard-deviation           2.15      1.88         2.86

COEFFICIENTS:
  export_fe: 2020 _ ABW 2020 _ AFG 2020 _ AGO 2020 _ AIA 2020 _ ALB
                  2.302    -0.7849       1.64     -4.887     -1.609
                   
  ... 673 remaining
-----
  import_fe: 2020 _ ABW 2020 _ AFG 2020 _ AGO 2020 _ AIA 2020 _ ALB
                  2.157     0.7903     -1.953     -1.152     -2.827
                   
  ... 673 remaining
-----
  bilateral_fe: ABW _ ARE ABW _ AUT ABW _ BEL ABW _ BRB ABW _ CAN
                    1.183     1.077   -0.9636      5.73    -11.96
                      
  ... 25,536 remaining
NULL


## HS2

### Problem getting FE --- следствие i^j^category

In [5]:
ComputeFE("../preprocessed_data/trade_hs2.parquet.gzip", 
                 is_category_aggregated = T, 
                 value_or_quantity = "value",
                 plain_or_change = 'plain',
                 save_path = "../preprocessed_data/trade_hs2",
                 interact_category = TRUE,
                 verbose = 100)

               Setup: 1.35s
           Demeaning: 1.39s (iter: 14)
          Estimation: 0.18s
     Post-processing: 0.44s
OLS estimation, Dep. Var.: log(v)
Observations: 1,935,761
Fixed-effects: export_fe: 48,921,  import_fe: 61,114,  bilateral_fe: 725,916
RMSE: 0.786325   Adj. R2: 0.924503
Problem getting FE, maximum iterations reached (1st order loop).

NOTE: The fixed-effects are not regular, they cannot be straightforwardly interpreted. The number of references is only approximate.



Fixed_effects coefficients
                        export_fe import_fe bilateral_fe
Number of fixed-effects     48921     61114       725916
Number of references            0        77         9948
Mean                         3.21     -1.01       -0.289
Standard-deviation           2.62      1.86         2.76

COEFFICIENTS:
  export_fe: 2020 _ ABW _ 03 2020 _ ABW _ 07 2020 _ ABW _ 15 2020 _ ABW _ 16
                       5.268          -1.537           4.672          -1.565
  2020 _ ABW _ 17                     
           -5.133 ... 48,916 remaining
-----
  import_fe: 2020 _ ABW _ 01 2020 _ ABW _ 02 2020 _ ABW _ 03 2020 _ ABW _ 04
                      -1.876          -1.192          -1.966          -0.996
  2020 _ ABW _ 05                     
           -2.468 ... 61,109 remaining
-----
  bilateral_fe: ABW _ ARE _ 22 ABW _ ARE _ 38 ABW _ ARE _ 40 ABW _ ARE _ 44
                        -2.899          2.545        -0.8632            1.3
  ABW _ ARE _ 70                      
      

In [ ]:
ComputeFE("../preprocessed_data/trade_hs2.parquet.gzip", 
                 is_category_aggregated = T, 
                 value_or_quantity = "quantity",
                 plain_or_change = 'plain',
                 save_path = "../preprocessed_data/trade_hs2",
                 interact_category = TRUE,
                 verbose = 100)

In [ ]:
ComputeFE("../preprocessed_data/trade_hs2.parquet.gzip", 
                 is_category_aggregated = T, 
                 value_or_quantity = "value",
                 plain_or_change = 'change',
                 save_path = "../preprocessed_data/trade_hs2",
                 interact_category = TRUE,
                 verbose = 100)

In [ ]:
ComputeFE("../preprocessed_data/trade_hs2.parquet.gzip", 
                 is_category_aggregated = T, 
                 value_or_quantity = "quantity",
                 plain_or_change = 'change',
                 save_path = "../preprocessed_data/trade_hs2",
                 interact_category = TRUE,
                 verbose = 100)

### HS2: i^j + category

In [4]:
ComputeFE("../preprocessed_data/trade_hs2.parquet.gzip", 
                 is_category_aggregated = T, 
                 value_or_quantity = "value",
                 plain_or_change = 'plain',
                 save_path = "../preprocessed_data/trade_hs2",
                 interact_category = FALSE,
                 verbose = 100)

               Setup: 0.66s
           Demeaning: 2.53s (iter: 30)
          Estimation: 0.11s
     Post-processing: 0.25s
OLS estimation, Dep. Var.: log(v)
Observations: 1,935,761
Fixed-effects: export_fe: 48,921,  import_fe: 61,114,  bilateral_fe: 25,779,  category: 96
RMSE: 1.99538   Adj. R2: 0.70293


NOTE: The fixed-effects are not regular, they cannot be straightforwardly interpreted. The number of references is only approximate.



Fixed_effects coefficients
                        export_fe import_fe bilateral_fe category
Number of fixed-effects     48921     61114        25779       96
Number of references            0       298          744       96
Mean                       -0.856    -0.408         -2.6     4.78
Standard-deviation           2.28      1.46          2.9     1.08

COEFFICIENTS:
  export_fe: 2020 _ ABW _ 03 2020 _ ABW _ 07 2020 _ ABW _ 15 2020 _ ABW _ 16
                       2.004       5.005e-07           1.484          -1.813
  2020 _ ABW _ 17                     
           -6.143 ... 48,916 remaining
-----
  import_fe: 2020 _ ABW _ 01 2020 _ ABW _ 02 2020 _ ABW _ 03 2020 _ ABW _ 04
                     -0.6648          0.4556         -0.3906           1.123
  2020 _ ABW _ 05                     
           -1.156 ... 61,109 remaining
-----
  bilateral_fe: ABW _ ARE ABW _ AUT ABW _ BEL ABW _ BRB ABW _ CAN
                    -5.34    -3.649    -7.629     0.602    -7.255
                    

In [ ]:
ComputeFE("../preprocessed_data/trade_hs2.parquet.gzip", 
                 is_category_aggregated = T, 
                 value_or_quantity = "quantity",
                 plain_or_change = 'plain',
                 save_path = "../preprocessed_data/trade_hs2",
                 interact_category = FALSE,
                 verbose = 100)

In [ ]:
ComputeFE("../preprocessed_data/trade_hs2.parquet.gzip", 
                 is_category_aggregated = T, 
                 value_or_quantity = "value",
                 plain_or_change = 'change',
                 save_path = "../preprocessed_data/trade_hs2",
                 interact_category = FALSE,
                 verbose = 100)

In [ ]:
ComputeFE("../preprocessed_data/trade_hs2.parquet.gzip", 
                 is_category_aggregated = T, 
                 value_or_quantity = "quantity",
                 plain_or_change = 'change',
                 save_path = "../preprocessed_data/trade_hs2",
                 interact_category = FALSE,
                 verbose = 100)